In [1]:
with open("input.txt") as f:
    lines = [
        line.strip() for line in f.readlines() if line.strip()
    ]

In [2]:
from typing import List

class Tile:

    def __init__(self, tile_id: int, grid: List[List[str]]):
        self.tile_id = tile_id
        self._grid = grid
        self._compute_lrtb()
        self.all_sides = {self.top, self.bottom, self.left, self.right}
        self.flip_horizontal()
        self.all_sides |= {self.top, self.bottom, self.left, self.right}
        self.flip_vertical()
        self.all_sides |= {self.top, self.bottom, self.left, self.right}
        self.flip_horizontal()
        self.all_sides |= {self.top, self.bottom, self.left, self.right}
        self.flip_vertical()

    def _compute_lrtb(self):
        """Sets the left, right, top, bottom based off of self._grid"""
        self.top = "".join(self._grid[0])
        self.bottom = "".join(self._grid[-1])
        self.left = "".join([line[0] for line in self._grid])
        self.right = "".join([line[-1] for line in self._grid])

    def rotate_clockwise(self):
        """Rotates the tile clockwise"""
        self._grid = list(zip(*self._grid[::-1]))
        self._grid = ["".join(line) for line in self._grid]
        self._compute_lrtb()

    def flip_horizontal(self):
        """Flips the tile along the vertical axis (i.e. left to right)"""
        self._grid = [line[::-1] for line in self._grid]
        self._compute_lrtb()

    def flip_vertical(self):
        """Flips along the horizontal axis"""
        self._grid = list(reversed(self._grid))
        self._compute_lrtb()

    def any_matches(self, other_tile) -> bool:
        """Returns whether any configuration allows two tiles to line up"""
        return bool(
            self.all_sides.intersection(other_tile.all_sides)
        )

    def __eq__(self, other_tile):
        return self.tile_id == other_tile.tile_id

    def __repr__(self):
        return str(self.name) + "\n" + "\n".join(self._grid)

In [3]:
from collections import defaultdict
import re

id_to_tile = defaultdict(list)
last_tile = None

for line in lines:

    if match := re.fullmatch("Tile (\d+):", line):
        last_tile = int(match.group(1))
    else:
        id_to_tile[last_tile].append(line)

id_to_tile = {
    tile_id: Tile(tile_id, grid) for tile_id, grid in id_to_tile.items()
}

## part1

We take a bit of a shortcut here. For each tile, we find all other tiles
that could possibly be a neighbor (must have two sides that could align).
If a tile only has 2 possible neighbors, then it must be a corner tile.

In [4]:
from functools import reduce
from operator import mul

tile_id_to_neighbors = {
    tile_id: [
        other_tile
        for other_tile in id_to_tile.values()
        if tile.any_matches(other_tile) and tile != other_tile
    ]
    for tile_id, tile in id_to_tile.items()
}

corners_ids = [
    tile_id
    for tile_id, neighbors in tile_id_to_neighbors.items()
    if len(neighbors) == 2
]
reduce(mul, corners_ids)

19955159604613